In [1]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt

In [3]:
# data prepratation
df = pd.read_csv('./data/co2-mm-mlo.csv', parse_dates=['Date'])
df = df.sort_values(by='Date').reset_index(drop=True)
df['time_idx'] = np.arange(len(df))
df['month'] = df['Date'].dt.month
y = df['Interpolated'].values

In [ ]:
with pm.Model() as bsts_model:
    # Hyperpriors for standard deviations -- Weakly Informative
    sigma_level = pm.HalfNormal("sigma_level", sigma=10.0)
    sigma_obs   = pm.HalfNormal("sigma_obs", sigma=10.0)
    
    # Local Trend Component: Gaussian Random Walk
     # Initial level (unregistered, separate from the random walk increments)
    mu0 = pm.Normal("mu0", mu=y[0], sigma=20.0)
    # Define increments for the random walk (for time steps 1 to T-1)
    mu_rw = pm.GaussianRandomWalk("mu_rw", sigma=sigma_level, shape=len(df)-1)
    # Construct the full latent level by concatenating mu0 with the cumulative sum of increments
    mu = pm.Deterministic("mu", pm.math.concatenate([[mu0], mu0 + pm.math.cumsum(mu_rw)]))
    
    # Define one seasonal parameter per month (12 in total) and impose a sum-to-zero constraint.
    seasonal_raw = pm.Normal("seasonal_raw", mu=0.0, sigma=10.0, shape=12)
    seasonal_effect = pm.Deterministic("seasonal_effect", seasonal_raw - pm.math.mean(seasonal_raw))
    
    # Map each observation to its corresponding monthly seasonal effect.
    month_effect = seasonal_effect[df['month'].values - 1]  # adjust for 0-indexing
    
    # The expected CO2 value is the sum of the latent trend and the seasonal effect.
    mu_obs = mu + month_effect
    
    # Observation model
    y_obs = pm.Normal("y_obs", mu=mu_obs, sigma=sigma_obs, observed=y)
    
    
    trace = pm.sample(2000, tune=2000, target_accept=0.9, return_inferencedata=True)

/Users/nika/code/kabitalama/MaunaLoa-BSTS-PyMC/.venv/lib/python3.12/site-packages/pymc/distributions/timeseries.py:297: UserWarning: Initial distribution not specified, defaulting to `Normal.dist(0, 100)`.You can specify an init_dist manually to suppress this warning.
  warnings.warn(


In [ ]:
# ----------------------------------------------------
# 2. Modified BSTS Model with a Local Linear Trend
# ----------------------------------------------------
with pm.Model() as bsts_local_linear:
    # Hyperpriors for noise terms
    sigma_level = pm.HalfNormal("sigma_level", sigma=10.0)
    sigma_slope = pm.HalfNormal("sigma_slope", sigma=1.0)
    sigma_obs   = pm.HalfNormal("sigma_obs", sigma=10.0)
    
    # Initial level and slope
    level0 = pm.Normal("level0", mu=y[0], sigma=20.0)
    slope0 = pm.Normal("slope0", mu=0.0, sigma=1.0)
    
    # Slope evolves as a Gaussian Random Walk (starting from slope0)
    # We simulate this by concatenating the initial slope with the random walk increments.
    rw_slope = pm.GaussianRandomWalk("rw_slope", sigma=sigma_slope, shape=len(df)-1)
    slope = pm.Deterministic("slope", pm.math.concatenate([[slope0], rw_slope]))
    
    # Level evolves according to the previous level plus the previous slope, plus error.
    # Compute the cumulative effect of the slope.
    level = pm.Deterministic("level", level0 + pm.math.cumsum(slope[:-1]))
    
    # Seasonal component (as before)
    seasonal_raw = pm.Normal("seasonal_raw", mu=0.0, sigma=10.0, shape=12)
    seasonal_effect = pm.Deterministic("seasonal_effect", seasonal_raw - pm.math.mean(seasonal_raw))
    month_effect = seasonal_effect[df['month'].values - 1]
    
    # The expected observation combines the level and seasonal effect.
    mu_obs = level + month_effect
    
    # Observation likelihood
    y_obs = pm.Normal("y_obs", mu=mu_obs, sigma=sigma_obs, observed=y[:-1])
    
    # Sample from the posterior
    trace_ll = pm.sample(2000, tune=2000, target_accept=0.9, return_inferencedata=True)
